Техническое задание содержится в файле
Документация.rar\ч. V Техничесая часть.docx

# 1. Поиск ТЗ в списке документации к закупке

In [10]:
# pip install bayoo-docx
# pip install aspose-zip
# pip install pdfplumber #Работа с .PDF

import os, shutil, sys, re
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import aspose.zip as az
import pdfplumber
import pandas as pd

def is_ascii(s):
    '''is_ascii(byte) -> bool

    Проверяет один байт на наличие в русском или англ. алфавитах,
    а так же среди знаков препинания.
    '''
    if (ord(s) in range(32, 122) ) or (ord(s) in range(1039, 1104))\
    or (ord(s) in (9,10,13)):
        return True
    return False

def doc2txt(ifname):
    with open(ifname, 'rb') as ffr:
        data = ffr.read()
    ## Определяет начало первого и конец последнего блоков текста
    start = data[:2800].rfind(b'\x00'*50) + 50
    end = start + data[start:].find(b'\x0d' + b'\x00'*10)
    if end < start: w95, end = True, start + data[start:].find(b'\x00'*10) ## Для 95 ворда
    else: w95 = False
    data = data[start:end]

    text, itemp = str(), 0

    ## Цикл читает получившийся текст по 2 байта и декодирует в читаемый вид
    while True:
        if w95 == False:
            try:
                t = data[itemp:itemp+2]
                if is_ascii(t.decode('utf-16le')) is True:
                    text += t.decode('utf-16le')

            except UnicodeDecodeError:
                t = t.decode('cp1251', 'ignore')
                if len (t) > 0:
                    for i in t:
                        if is_ascii(i) is True:
                            text += i
            itemp += 2
            if itemp >= len(data): break

    ## Если это кодировка cp1251, то декодирует весь текст в один прием
        else:
            t = data.decode('cp1251', 'ignore')
            for i in t:
                if is_ascii(i) is True:
                    text += i
            break

    return text #.encode()

def zip2dir(fname):
    # Загрузить архив
    with az.Archive(fname) as archive:
	    # Извлечь файл 
        fn = os.path.basename(fname)
        archive.extract_to_directory(os.path.join(path_temp, fn))

def rar2dir(fname):
    # Загрузить архив
    with az.rar.RarArchive(fname) as archive:
        # Извлечь файл 
        fn = os.path.basename(fname)
        archive.extract_to_directory(os.path.join(path_temp, fn))


def iter_block_items(parent):
    """
    Generate a reference to each paragraph and table child within *parent*,
    in document order. Each returned value is an instance of either Table or
    Paragraph. *parent* would most commonly be a reference to a main
    Document object, but also works for a _Cell object, which itself can
    contain paragraphs and tables.
    """
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    elif isinstance(parent, _Row):
        parent_elm = parent._tr
    else:
        raise ValueError("something's not right")
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

def docx2txt(docx_file):
    document = Document(docx_file)
    text = []
    for block in iter_block_items(document):
        #print(block.text if isinstance(block, Paragraph) else '<table>')
        if isinstance(block, Paragraph):
            if len(block.text.strip()) > 0:
                text.append(block.text)
        elif isinstance(block, Table):
            for row in block.rows:
                row_data = []
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                        row_data.append(paragraph.text)
                text.append("\t".join(row_data))    
    return '\n'.join(text)

def pdf2txt(fname):
    with pdfplumber.open(fname) as pdf:
        if len(pdf.pages):
            text_o = ' '.join([
                page.extract_text() or '' for page in pdf.pages if page
            ])
    return text_o

def xlsx2txt(fname):
    return pd.read_excel(fname).to_string()

def to_txt(path):
    txt_from_files = {}
    for root, dirs, files in os.walk(path):
        #print(f'{root=}, {dirs=}, {files=}')
        for fn in files:
            ext = fn.split('.')[-1]
            ffn = os.path.join(root, fn)
            if ext=='doc':
                txt_from_files[ffn] = doc2txt(ffn)
            elif ext=='docx':
                txt_from_files[ffn] = docx2txt(ffn)
            elif ext=='txt':
                with open(ffn,'r') as f:
                    txt_from_files[ffn] = f.read()
            elif ext in [ 'xlsx']:
                txt_from_files[ffn] = xlsx2txt(ffn)
            elif ext=='rar':
                rar2dir(ffn)
            elif ext=='zip':
                zip2dir(ffn)
    return txt_from_files

def clear_temp(path_temp):# Очистка временной папки
    if os.path.exists(path_temp): 
        shutil.rmtree(path_temp)
    os.mkdir(path_temp)

def find_nearest_file(texts, keywords):
    '''
    На вход подается словарь вида имя файла: текст и список ключевых фраз в формате re
    Функция ищет по ключевым фразам в тексте наиболее близкий по содержанию файл
    На выходе имя найденного файла 
    '''
    closest_file = None
    max_count = 0
    for filename, text in texts.items():
        if len(text)>0: 
            if sum([len(re.findall (k, os.path.basename(filename).lower())) for k in keywords]): # Если найдены совпадения в названии файла
                return filename
            else:
                text = re.sub("[^а-я\n ]", "", text.lower()).strip() # оставляем только русские буквы и перевод строки
                cnt = sum([len(re.findall (k, text)) for k in keywords])
                distance = 0
                text = text.split('\n')
                if cnt: # если найдено хоть одно совпадение
                    for i,s in enumerate(text): # то ищем номера строк в которых эти ключи найдены
                        cnt = sum([len(re.findall (k, s)) for k in keywords])
                        if cnt:
                            distance += cnt/(i+1) # вычисляем дистанцию, чем дальше фраза от начала документа, тем меньше ее вес
                    if distance > max_count:
                        max_count = distance
                        closest_file = filename
                print(f'{distance=:0.3f}, {filename=}')
                    
    return closest_file

# ==================================================================
path_root = "C:/temp/"
path_temp = "temp"
keywords = [r"\bтехническ.. задан*", r"\bтехническ.. описан*"]

if  not os.path.exists(path_root):
    print("Папка не существует")
    sys.exit()

_, dirs, _ in next(os.walk(path_root))

dirs = ['0128200001624000002']
for path in dirs:
    clear_temp(path_temp)
    path = os.path.join(path_root, path)    
    txt_from_files = to_txt(path)
    txt_from_arch = to_txt(path_temp)
    if txt_from_arch:
        txt_from_files = txt_from_files | txt_from_arch
    #clear_temp(path_temp)
    file_tz = find_nearest_file(txt_from_files, keywords)
    print(f'Техническое задание из каталога {path} с наибольшей вероятностью содержится в файле: {file_tz}')

# for key, value in txt_from_files.items():
#     if len(value)!=0:
#         fn = os.path.basename(key)
#         fn = '.'.join(fn.split('.')[:-1]) +'.txt'
#         with open(os.path.join(path_temp, fn), 'w', encoding='utf-8') as f:
#             f.write(value)


distance=0.000, filename='C:/temp/0128200001624000002\\docs\\Приложение 1 обоснование НМЦК_ЭА.docx'
distance=0.000, filename='C:/temp/0128200001624000002\\docs\\Приложение 3 Требование к содержанию заявки.docx'
Техническое задание из каталога C:/temp/0128200001624000002 с наибольшей вероятностью содержится в файле: C:/temp/0128200001624000002\docs\Приложение №2 Техническое задание.docx


In [8]:
_, dirs, _ in next(os.walk(path_root))
dirs

['0128200001624000002']